In [1]:
!pip install kafka-python
!pip install redis

In [2]:
import os
import requests
import json
import redis
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[2] pyspark-shell'
conf = SparkConf().set("spark.jars","/home/jovyan/work/spark-streaming-kafka-0-8-assembly_2.11-2.4.4.jar")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,60) # 60 is the batch interval :)
debug = True
saveToLocalDisk = True
spark = SparkSession(sc)

In [4]:
def SetVal(x):
    r = redis.StrictRedis(host = 'redis-store', port = 6379)
    r.set(x,int(x))
    
def SaveToFile(rdd):
    if not rdd.isEmpty():
        rdd.toDF( [ "Ticker", "Timestamp", "Price" ] ) \
        .write.save("/home/jovyan/work/FinanceData.csv",
                    maxRecordsPerFile=1000, format="csv", mode="append") 

In [5]:
kafkaStream = KafkaUtils.createStream(
    ssc=ssc, 
    zkQuorum='zookeeper:2181', 
    groupId='test-consumer-group',#, #Consumer Group 
    topics={'finance_ticker':1})

olines = kafkaStream.map(lambda x: x[1]).map(lambda a: json.loads(a))
if(debug):
    olines.pprint()
if(saveToLocalDisk):
    olines.map(lambda a: (a['Ticker'],a['Timestamp'],a['Price'])) \
    .foreachRDD(SaveToFile)
    
lines = olines.map(lambda a: str(a['Price'])) \
        .reduce(lambda v,agg: agg+", "+v)
if(debug):
    lines.pprint()

lines = lines.map(lambda a: requests.post(
        "http://serving:8501/v1/models/half_plus_two:predict",
        "{\"instances\": ["+a+"]}").text)
if(debug):
    lines.pprint()

lines = lines.flatMap(lambda x: json.loads(x)["predictions"])
if(debug):
    lines.pprint()

lines = lines.map(lambda x: SetVal(x))
if(debug):
    lines.pprint()

ssc.start()  
print("Going to wait termination")
ssc.awaitTermination()

Going to wait termination
-------------------------------------------
Time: 2019-11-24 21:03:00
-------------------------------------------

-------------------------------------------
Time: 2019-11-24 21:03:00
-------------------------------------------

-------------------------------------------
Time: 2019-11-24 21:03:00
-------------------------------------------

-------------------------------------------
Time: 2019-11-24 21:03:00
-------------------------------------------

-------------------------------------------
Time: 2019-11-24 21:03:00
-------------------------------------------

-------------------------------------------
Time: 2019-11-24 21:04:00
-------------------------------------------
{'Ticker': 'MSFT', 'Timestamp': 1574456401, 'Price': 149.59}
{'Ticker': 'AAPL', 'Timestamp': 1574456401, 'Price': 261.78}

-------------------------------------------
Time: 2019-11-24 21:04:00
-------------------------------------------
261.78, 149.59

--------------------------------

KeyboardInterrupt: 